In [10]:
#!pip install deep-hedging

In [11]:
import datetime as dt

import numpy as np

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    EuropeanCall,
)
from deep_hedging.monte_carlo import GBMPricer

RANDOM_SEED = 12

In [12]:
TILL_MATURITY = 3.0
N_DAYS = 252 * TILL_MATURITY

LOWER_STRIKE = 100 / 100
UPPER_STRIKE = 112 / 100
PARTICIPATION = 2

RF_RATE = 0.03
VOL = 0.2

In [13]:
DESIRED_PAYOFF = lambda spot: PARTICIPATION * np.maximum(
    0, np.minimum(spot[:, -1] - 1, UPPER_STRIKE - LOWER_STRIKE)
)

In [14]:
start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

underlyings = Underlyings(
    tickers=[Ticker(f"Some Stock", f"STNK")],
    start=start,
    end=end,
    dividends=np.array([0.0]),
    means=np.array([RF_RATE]),
    var_covar=np.array([VOL**2]),
)
underlyings.data

In [15]:
curve = ConstantRateCurve(constant_rate=RF_RATE)

In [16]:
call_lower = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=LOWER_STRIKE,
    start_date=start,
    end_date=end,
)
call_upper = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=UPPER_STRIKE,
    start_date=start,
    end_date=end,
)

call_spread = (call_lower - call_upper) * PARTICIPATION
call_spread

StructuredNote of:
1. LONG 2.0 units of EuropeanCall:
* Underlyings:
-> Ticker(name='Some Stock', code='STNK', currency=None)
* Strike = 100.0
* Start Date = 2019-01-01 00:00:00
* End Date = 2021-01-26 00:00:00
.

2. SHORT 2.0 units of EuropeanCall:
* Underlyings:
-> Ticker(name='Some Stock', code='STNK', currency=None)
* Strike = 112.00000000000001
* Start Date = 2019-01-01 00:00:00
* End Date = 2021-01-26 00:00:00
.


In [17]:
pricer = GBMPricer(payoff_function=call_spread.payoff, random_seed=RANDOM_SEED)
paths = pricer.get_paths(
    spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_instant_fwd_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2] * len(t)),
).squeeze(2)
paths.shape

(100000, 757)

In [18]:
assert np.allclose(call_spread.payoff(paths)[:, -1], DESIRED_PAYOFF(paths), atol=1e-6)